Question 1
Create the following tables as per cardinality:

student: rollno, name, address, dob, gender, percentage, classid

class: classid, classname

One class can have many students.

Insert 5 records in each dataframe and perform the following tasks:

In [11]:
import findspark
findspark.init()
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, year, when, concat_ws, upper, lower, avg, sum, count, udf, max, min,concat_ws, upper, lower, length, substring, instr, trim, lpad, rpad
from pyspark.sql.types import StringType, IntegerType, DoubleType


In [2]:
# Initialize Spark session
spark = SparkSession.builder.appName("Student_Class").getOrCreate()

In [3]:
# Student data
student_data = [
    (1, 'Amit', 'Pune', '2002-04-10', 'M', 78, 101),
    (2, 'Bhavna', 'Mumbai', '2001-08-15', 'F', 85, 101),
    (3, 'Chetan', 'Nashik', '2003-09-09', 'M', 91, 102),
    (4, 'Deepa', 'Pune', '2002-12-29', 'F', 68, 102),
    (5, 'Esha', 'Nagpur', '2001-11-21', 'F', 72, 101)
]

student_columns = ['rollno', 'name', 'address', 'dob', 'gender', 'percentage', 'classid']
student = spark.createDataFrame(student_data, student_columns)

In [4]:
# Class data
class_data = [
    (101, 'FYBSc'),
    (102, 'SYBSc')
]

class_columns = ['classid', 'classname']
class_df = spark.createDataFrame(class_data, class_columns)

This creates two dataframes—students and classes—with appropriate columns and sample data, establishing the ‘one class to many students’ relationship.

2. Print the schema of both dataframes.

In [5]:
student.printSchema()
class_df.printSchema()

root
 |-- rollno: long (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- percentage: long (nullable = true)
 |-- classid: long (nullable = true)

root
 |-- classid: long (nullable = true)
 |-- classname: string (nullable = true)



In [6]:
student.show()

+------+------+-------+----------+------+----------+-------+
|rollno|  name|address|       dob|gender|percentage|classid|
+------+------+-------+----------+------+----------+-------+
|     1|  Amit|   Pune|2002-04-10|     M|        78|    101|
|     2|Bhavna| Mumbai|2001-08-15|     F|        85|    101|
|     3|Chetan| Nashik|2003-09-09|     M|        91|    102|
|     4| Deepa|   Pune|2002-12-29|     F|        68|    102|
|     5|  Esha| Nagpur|2001-11-21|     F|        72|    101|
+------+------+-------+----------+------+----------+-------+



In [7]:
class_df.show()

+-------+---------+
|classid|classname|
+-------+---------+
|    101|    FYBSc|
|    102|    SYBSc|
+-------+---------+



Displays the structure and data types of each column allowing us to verify data correctness.

3.  Display student names with gender ‘M’ and ‘F’ separately.

In [8]:
print("Male students:")
student.filter(col("gender") == 'M').select("name").show()

print("Female students:")
student.filter(col("gender") == 'F').select("name").show()


Male students:
+------+
|  name|
+------+
|  Amit|
|Chetan|
+------+

Female students:
+------+
|  name|
+------+
|Bhavna|
| Deepa|
|  Esha|
+------+



Filters allow us to analyze gender-based distribution of students.

4. Display students of a particular class, e.g., classid = 101.

In [9]:
student.filter(col("classid") == 101).show()


+------+------+-------+----------+------+----------+-------+
|rollno|  name|address|       dob|gender|percentage|classid|
+------+------+-------+----------+------+----------+-------+
|     1|  Amit|   Pune|2002-04-10|     M|        78|    101|
|     2|Bhavna| Mumbai|2001-08-15|     F|        85|    101|
|     5|  Esha| Nagpur|2001-11-21|     F|        72|    101|
+------+------+-------+----------+------+----------+-------+



Selects all students belonging to a specific class.

5. Calculate age from date of birth and show students aged 22 (assuming current year 2025).

In [12]:
student_with_age = student.withColumn("age", 2025 - year(col("dob")))
student_with_age.filter(col("age") == 22).show()


+------+------+-------+----------+------+----------+-------+---+
|rollno|  name|address|       dob|gender|percentage|classid|age|
+------+------+-------+----------+------+----------+-------+---+
|     3|Chetan| Nashik|2003-09-09|     M|        91|    102| 22|
+------+------+-------+----------+------+----------+-------+---+



Calculates age and filters students by age group for demographic insights.

6. Show classnames associated with their students.

In [13]:
student.join(class_df, "classid").select("classname", "name").show()


+---------+------+
|classname|  name|
+---------+------+
|    FYBSc|  Amit|
|    FYBSc|Bhavna|
|    FYBSc|  Esha|
|    SYBSc|Chetan|
|    SYBSc| Deepa|
+---------+------+



Joins tables to combine student and class information for comprehensive reporting.

7. Fetch data of students born in 2001.

In [14]:
student.filter(year(col("dob")) == 2001).show()


+------+------+-------+----------+------+----------+-------+
|rollno|  name|address|       dob|gender|percentage|classid|
+------+------+-------+----------+------+----------+-------+
|     2|Bhavna| Mumbai|2001-08-15|     F|        85|    101|
|     5|  Esha| Nagpur|2001-11-21|     F|        72|    101|
+------+------+-------+----------+------+----------+-------+



Filters students by birth year, useful for cohort analysis.

8. Create a UDF to assign letter grades based on percentage.
python

In [15]:
def grade(percentage):
    if percentage >= 90:
        return 'A+'
    elif percentage >= 80:
        return 'A'
    elif percentage >= 70:
        return 'B'
    elif percentage >= 60:
        return 'C'
    else:
        return 'D'

grade_udf = udf(grade, StringType())
student_with_grade = student.withColumn("grade", grade_udf(col("percentage")))
student_with_grade.select("name", "percentage", "grade").show()


+------+----------+-----+
|  name|percentage|grade|
+------+----------+-----+
|  Amit|        78|    B|
|Bhavna|        85|    A|
|Chetan|        91|   A+|
| Deepa|        68|    C|
|  Esha|        72|    B|
+------+----------+-----+



Adds a computed column ‘grade’ by categorizing percentages into letter grades.